In [259]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.corpus import stopwords
import re
import nltk
import csv
from nltk.stem.snowball import SnowballStemmer

In [260]:
reviewsFrame = pd.read_csv('Reviews.csv', nrows=2000)

In [262]:
def remove_punctuation(reviewStr):
    reviewStr = re.sub(r"[,.;@#?!&$-:/'_]+\ *", " ", reviewStr)
    reviewStr = reviewStr.replace('"','')
    reviewStr = reviewStr.replace('\\','')
    return reviewStr
    

In [263]:
def remove_stop_words(reviewStr):
    stopset = stopwords.words('english')
    #stopset.extend(["even","ever","","wan","na","www","whose","html","all","you","your","i","but","every","also","got","must","still","going","it","want","yup","wassup","without"])
    sep = " "
    return sep.join(filter(lambda x: x not in stopset, reviewStr.lower().split()))

In [264]:
def remove_pos_stopwords(reviewStr):
    STOP_TYPES = ['DET', 'CNJ']
    tokens = nltk.pos_tag(nltk.word_tokenize(reviewStr))
    return " ".join([w for w, wtype in tokens if wtype not in STOP_TYPES])



In [265]:
def remove_stems(reviewStr):
    ps = SnowballStemmer("english")
    reviewStr = " ".join([ps.stem(word) for word in reviewStr.split(" ")])
    return reviewStr

In [266]:
reviewsFrame['ProcessedText'] = reviewsFrame['Text'].apply(remove_punctuation)

In [268]:
reviewsFrame['ProcessedText'] = reviewsFrame['ProcessedText'].apply(remove_stop_words)

In [270]:
reviewsFrame['ProcessedText'] = reviewsFrame['ProcessedText'].apply(remove_pos_stopwords)


In [272]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words = 'english')

In [273]:
tfidfMat = vectorizer.fit_transform(reviewsFrame['ProcessedText'])

In [274]:
feature_names = vectorizer.get_feature_names() 

In [276]:
dense= tfidfMat.todense()

In [248]:
with open("tfidf_scikit.csv", "w") as file:
    writer = csv.writer(file, delimiter=",")
    writer.writerow(["ReviewId","Word", "Score"])
    
    doc_id = 0
    for doc in dense:
        word_id = 0
        for score in doc.tolist()[0]:  
            if score > 0 :
                word = feature_names[word_id]
                writer.writerow([doc_id+1, word.encode("utf-8"), score])
            word_id +=1
        doc_id +=1